In [3]:
import sys
import os
if not any(path.endswith('textbook') for path in sys.path):
    sys.path.append(os.path.abspath('../../..'))
from textbook_utils import *

csv_file = 'data/Full24hrdataset.csv'
usecols = ['Date', 'ID', 'region', 'PM25FM', 'PM25cf1']
full = (pd.read_csv(csv_file, usecols=usecols, parse_dates=['Date'])
        .dropna())
full.columns = ['date', 'id', 'region', 'pm25aqs', 'pm25pa']

GA = full.loc[(full['id'] == 'GA1')  , :]

In [24]:
outlier_czs = [34105, 34113, 34112, 34106]
df = (
    pd.read_csv('data/mobility.csv')
    # filter out rows with NaN AUM values
    .query('not aum.isnull()', engine='python')
    # take out outlier CZs
    .query('cz not in @outlier_czs')
)

(sec:linear_simple_fit)=
# Fitting the Simple Linear Model

To fit a model, we use the notion of loss minimization that was first introduced in
{numref}`Chapter %s <ch:modeling>`).
The first step is to choose the loss function. Then, we find which values of
$\theta_0$ and $\theta_1$ give the smallest loss for our data.
We call $\theta_0$ and $\theta_1$ *model parameters*, and we often represent them as  ${\boldsymbol{\theta}} = [\theta_0, \theta_1]$.

To fit a linear model with a numeric response, we typically choose squared loss, where for a given data point $(x,y)$ and model parameters $\underline{\mathbf{\theta}}$, the squared loss is:

$$
{\cal l}(\underline{\mathbf{\theta}}, x, y) = [y - (\theta_0 + \theta_1 x)]^2
$$

Notice that this loss function squares the error between the observed $y$ and the point on the line, $\theta_0 + \theta_1 x$. That's why squared loss is also called *squared error*. We saw in the previous section that these errors are in vertical direction, meaning for a specific $x$, the error is the vertical difference between the points $(x, y)$ and $(x, \theta_0 + \theta_1 x)$.  (See {numref}`Figure %s <fig:SLR>` for a diagram that includes two sample errors). 

For a data set with $n$ points: $(x_1, y_1), \ldots, (x_n, y_n) $, the average squared loss (ASE) is:

$$
\begin{aligned}
L(\underline{\mathbf{\theta}}, \underline{\mathbf{x}}, \underline{\mathbf{y}})
 &= \frac{1}{n} \sum_{i} {\cal l}(\underline{\mathbf{\theta}}, x_i, y_i) \\
 &= \frac{1}{n} \sum_{i}[y_i - (\theta_0 + \theta_1 x_i)]^2.
\end{aligned}
$$

Here $\underline{\mathbf{x}}$ is the vector $[x_1, \ldots, x_n]$ and $\underline{\mathbf{y}}$ is similarly defined.

Now that we have chosen the loss function, we want to find $\hat{\underline{\mathbf{\theta}}}$ that minimizes the loss. We use calculus to do this.

## Minimizing the Loss

With the simple linear model, the average squared error is a function of two model parameters. This means that if we use calculus to find the minimizing parameter values, we need to find the partial derivatives of the ASE with respect to $\theta_0$ and $\theta_1$. We can also find these minimizing values using other techniques:

+ *Gradient descent*. We use numerical optimization techniques, such as gradient descent, in later chapters when the loss function is more complex and calculus is not an option.
+ *Quadratic formula*. Since $L(\underline{\mathbf{\theta}}, \underline{\mathbf{x}}, \underline{\mathbf{y})}$ is a quadratic function of $\underline{\mathbf{\theta}}$ we can use the quadratic formula (along with a lot of algebra) to solve for the minimizing model parameter values. We guide you through the quadratic approach in the exercises. 
+ *Geometric argument*. Later in this chapter, we use a geometric interpretation of least squares to fit multiple linear models. This approach relates to the Pythagorean theorem and has several intuitive benefits.

We chose to use calculus for solving the simple linear model since it is quick and straightforward. To begin, we take the partial derivative of the average squared error with respect to each parameter:
 
$$
\begin{aligned}
\frac{\partial}{\partial \theta_0} L(\underline{\mathbf{\theta}}, \underline{\mathbf{x}}, \underline{\mathbf{y}})
  &= \frac{1}{n} \sum_{i} 2 (y_i - \theta_0 - \theta_1 x_i ) (-1)\\
 & \\ 
\frac{\partial}{\partial \theta_1} L(\underline{\mathbf{\theta}}, \underline{\mathbf{x}}, \underline{\mathbf{y}})
  &= \frac{1}{n} \sum_{i} 2 (y_i - \theta_0 - \theta_1 x_i) (-x_i)  
\end{aligned}
$$

Then, we set the partial derivatives equal to 0, and simplify a bit by multiplying both sides of the equations by $-n/2$ to get:

$$
\begin{aligned}
 0   &= \sum_{i} (y_i - \hat{\theta}_0 - \hat{\theta}_1 x_i) \\
 0   &= \sum_{i} (y_i - \hat{\theta}_0 - \hat{\theta}_1 x_i)x_i \\
\end{aligned}
$$

These equations are called the *normal equations*. 
In the first equation, we see that $\hat{\theta}_0$ can be represented as a function of $\hat{\theta}_1$.

$$
\hat{\theta}_0 = \bar{y} - \hat{\theta}_1 \bar{x}
$$

Plugging this value into the second equation, gives us:

$$
\begin{aligned}
 0   &= \sum_{i} (y_i - \bar y + \hat{\theta}_1 \bar x - \hat{\theta}_1 x_i ) x_i \\
  &= \sum_{i} [(y_i - \bar y) - \hat{\theta}_1 ( x_i - \bar x)]x_i \\ 
\hat{\theta}_1  &= \frac{\sum_{i} (y_i - \bar y)x_i} {\sum_{i}( x_i - \bar x)x_i} \\
\end{aligned}
$$

After some algebra, we can represent $\hat{\theta}_1$ in terms of quantities that we are familiar with:

$$
\hat{\theta}_1 = r(\underline{\mathbf{x}},\underline{\mathbf{y}}) \frac{SD(\underline{\mathbf{y}})}{SD(\underline{\mathbf{x}})}
$$

Using this representation, a point on the fitted line at $x$ can be written as

$$ 
\begin{aligned}
\hat{y} &= \hat{\theta}_0 + \hat{\theta}_1 x \\
&= \bar{y} - \hat{\theta}_1 \bar{x} + \hat{\theta}_1 x \\
&= \bar{y} + r(\underline{\mathbf{x}},\underline{\mathbf{y}}) SD(\underline{\mathbf{y}}) \frac{(x - \bar{x})}{SD(\underline{\mathbf{x}})}\\ 
\end{aligned}
$$

This form of the least squares line has a nice interpretation: for a given $x$ value, we find how many standard deviations above (or below) average it is, and then predict $y$ to be $r$ times as many standard deviations above (or below) its average. 

Since `pandas.Series` objects have built-in methods to compute
$SD(x)$, $SD(y)$, and $r$,
we can quickly define functions that fit the simple linear model to our data:

In [4]:
def theta_1(x, y):
    r = x.corr(y)
    return r * y.std() / x.std()

In [5]:
def theta_0(x, y):
    return y.mean() - theta_1(x, y) * x.mean()

Now, we can fit the model by computing
$\hat{\theta}_0$ and $\hat{\theta}_1$ on the data set.

In [6]:
t1 = theta_1(GA['pm25aqs'], GA['pm25pa'])
t0 = theta_0(GA['pm25aqs'], GA['pm25pa'])
print(f'Model: f(x) = {t0:.2f} + {t1:.2f}x')

Model: f(x) = -3.36 + 2.10x


The model we fitted matches the line that we used in the previous section. Of course, we will want to use the functionality provided in scikit learn because it offers numerically stable algorithms. This is especially important when fitting multiple variables.

In [15]:
from sklearn.linear_model import LinearRegression

y = GA[['pm25pa']]
X = GA[['pm25aqs']]

reg = LinearRegression().fit(X, y)

In [43]:
print(f"Intercept: {reg.intercept_}  Slope: {reg.coef_[0]}")

Intercept: [-3.36]  Slope: [2.1]


In this section, we derived the model parameters
$ \hat{\theta}_0 $ and $ \hat{\theta}_1 $ using calculus.
Then, we fit a simple linear model that predicts PurpleAir moitor readings
from readings on a more accurate sensor.  
A residual plot shows that this linear model fits the data well.
The scientists studying the monitors were interested in further improving the model, by considering weather factors. That is they were interested in whether or not
multiple predictor variables, rather than just one, would create a better model.
When multiple variables are used as explanatory variables in a model, we called it a *multiple linear model*. We introduce this model in the next section.